## Make sure the Ollama Served model is accessible

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel

In [ ]:
remote_url = "http://{}:11434"
model = LiteLLMModel(
    base_url=remote_url
    , model_id='ollama/qwen2.5-coder:7b'
    # , model_id='ollama/llama3.2:latest'
)

In [ ]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)

In [ ]:
agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

## Using a Custom Tool to Prepare the Menu
Now that we have selected a playlist, we need to organize the menu for the guests. Again, Alfred can take advantage of smolagents to do so. Here, we use the `@tool` decorator to define a custom function that acts as a tool. We'll cover tool creation in more detail later, so for now, we can simply run the code.

As you can see in the example below, we will create a tool using `@tool` decorator and include it in the tools list.

In [ ]:
from smolagents import CodeAgent, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if "casual" in occasion.lower():
        return "Pizza, snacks, and drinks."
    elif "formal" in occasion.lower():
        return "3-course dinner with wine and dessert."
    elif "superhero" in occasion.lower():
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

menu_agent = CodeAgent(tools=[suggest_menu], model=model)

In [ ]:
menu_agent.run("Prepare a formal menu for the party.")

## Using Python Imports Inside the Agent
We have the playlist and menu ready, but we need to check one more crucial detail: preparation time!

Alfred needs to calculate when everything would be ready if he started preparing now, in case they need assistance from other superheroes.

`smolagents` specializes in agents that write and execute Python code snippets, offering sandboxed execution for security. It supports both open-source and proprietary language models, making it adaptable to various development environments.

Code execution has strict security measures - imports outside a predefined safe list are blocked by default. However, you can authorize additional imports by passing them as strings in `additional_authorized_imports`. For more details on secure code execution, see the official [guide](https://huggingface.co/docs/smolagents/tutorials/secure_code_execution).

When creating the agent, we ill use `additional_authorized_imports` to allow for importing the `datetime` module.

In [ ]:
from smolagents import CodeAgent
# import numpy as np
# import time
# import datetime

agent = CodeAgent(tools=[], model=model, additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    3. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

These examples are just the beginning of what you can do with code agents, and we're already starting to see their utility for preparing the party. You can learn more about how to build code agents in the [smolagents documentation](https://huggingface.co/docs/smolagents).

`smolagents` specializes in agents that write and execute Python code snippets, offering sandboxed execution for security. It supports both open-source and proprietary language models, making it adaptable to various development environments.

## Sharing Our Custom Party Preparator Agent to the Hub

Wouldn't it be amazing to share our very own Alfred agent with the community? By doing so, anyone can easily download and use the agent directly from the Hub, bringing the ultimate party planner of Gotham to their fingertips! Let's make it happen! 🎉

The `smolagents` library makes this possible by allowing you to share a complete agent with the community and download others for immediate use. It's as simple as the following:

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, VisitWebpageTool, FinalAnswerTool
from my_tools import suggest_menu, catering_service_tool, SuperheroPartyThemeTool
# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool(),
        FinalAnswerTool()
        ],
    model=model,
    max_steps=10,
    verbosity_level=2
)
agent.run("""
    Give me theh best list songs for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme.
    The output should be a list of song names and artists.
""")


### Now we can push this agent to huggingface hub to share accross

In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
for tool in agent.tools:
    print(tool)
    # print(tool.code)  # or tool.to_dict()["code"]


In [ ]:
agent.push_to_hub('mlops-pltf/AlfredAgent')

### To reuse the agent:

In [ ]:
agent = CodeAgent(tools=[], model=model)
alfred_agent = agent.from_hub('mlops-pltf/AlfredAgent', trust_remote_code=True)

In [ ]:
alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'classic heroes' theme")